In [1]:
import json
import pandas as pd

with open('../data/triviaqa-unfiltered/unfiltered-web-dev.json','r') as f:
    data = json.loads(f.read()) # Flatten data
df = pd.json_normalize(data, record_path =['Data'])
df = df[['Question', 'QuestionId', 'QuestionSource', 'Answer.Aliases', 'Answer.Value']]
df

,Question,QuestionId,QuestionSource,Answer.Aliases,Answer.Value
0,Who was the man behind The Chipmunks?,tc_2,http://www.triviacountry.com/,[David Seville],David Seville
1,What star sign is Jamie Lee Curtis?,tc_13,http://www.triviacountry.com/,"[Scorpio, Skorpio, Scorpio (disambiguation)]",Scorpio
2,Which Lloyd Webber musical premiered in the US...,tc_33,http://www.triviacountry.com/,"[Sunset Blvd, West Sunset Boulevard, Sunset Bo...",Sunset Boulevard
3,Who was the next British Prime Minister after ...,tc_40,http://www.triviacountry.com/,"[Sir Henry Campbell-Bannerman, Campbell-Banner...",Campbell-Bannerman
4,Who had a 70s No 1 hit with Kiss You All Over?,tc_49,http://www.triviacountry.com/,"[Internal exile, Exiles, Transported for life,...",Exile
...,...,...,...,...,...
11308,Who tried to steal Christmas from the town of ...,qg_4615,https://quizguy.wordpress.com/,"[Grinch, The grinch, The Grinch]",The Grinch
11309,"According to the proverb, what comes but once ...",qg_4620,https://quizguy.wordpress.com/,"[Orthodox Christmas, Holiday decorations, Chir...",Christmas
11310,What is the name of the parson mentioned in th...,qg_4628,https://quizguy.wordpress.com/,"[Orangeish blacks, Light brown, Orangeish blac...",Brown
11311,"According to a popular Christmas song, who was...",qg_4629,https://quizguy.wordpress.com/,"[Frosty The Snowman, Frosty and Rudolph's Chri...",Frosty the Snowman.


In [2]:
import os
import openai

openai.api_key = '' # <YOUR_KEY>

def translate_to_sparql(text):
    prompt = f"Turn this into a Wikidata SPARQL query: {text}"
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        temperature=0.5,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response["choices"][0]["text"]

text = "What instrument does Kermit the Frog play?"
sparql_query = translate_to_sparql(text)
print(sparql_query)



SELECT ?instrument WHERE { wd:Q237220 wdt:P1303 ?instrument. }


In [3]:
from tqdm import tqdm
import time

sample = df.sample(n=100)

def ask_q(text):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=text,
        temperature=0.5,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response["choices"][0]["text"]

answers = []
for q in tqdm(sample['Question']):
    
    answers.append(ask_q(q))
    time.sleep(2)

sample['Answer.GPT3'] = answers

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [04:27<00:00,  2.67s/it]


In [4]:
sample['Answer.GPT3'] = sample['Answer.GPT3'].apply(str.strip, args=('?\n',))

In [6]:
sample[['Question', 'Answer.Value', 'Answer.GPT3']].to_csv('sample_answers.csv')